# 8.2 Web Scraping I.


Web-Scrapping es la forma que tenemos para referirnos a la captura de información de cualquier sitio web. Su objetivo es capturar información de forma automática.

Las librerías principales que vamos a utilizar son beautifulsoup y requests


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

El proceso se divide principalmente en tres fases:

- Carga de la dirección web a la que realizar el scrapping. A través de  requests.

- Extracción del contenido de la web a partir de Beautifulsoup

- Manipulación del contenido.

- Para tablas podemos extraer directamente el contenido en pandas.


## 1. Carga de la URL

Con requests podemos obtener el contenido de la pagina web.

In [ ]:
url = 'https://es.wikipedia.org/wiki/El_lobo_de_Wall_Street'
r = requests.get(url)

Ahora debemos preguntar si la conexion con esa pagina web ha funcionado. Es decir si la propiedad status_code que devuelve re es 200 o 201. Si es así, pasamos a la siguiente fase.

In [ ]:
print(r.status_code)

En re.content tendremos todo el HTML

In [ ]:
r.content

## 2. Extracción del contenido con Pandas

Si usamos la función read_html sobre una URL. Nos data una lista de Dataframes con todas las tablas de la página.
```python
    dfs = pd.read_html(url)
``` 

- Esta función nos da la opcion de filtrar con el parámetro match.

In [ ]:
dfs = pd.read_html('https://www.formula1.com/en/results.html/2023/drivers.html')

In [ ]:
dfs[0]

## 3. Extracción del contenido con BeautifulSoup

Lo primero que tenemos que hacer es pasar al contenido que se encuentra en re.text a través de un parser de HTML.

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

Lo siguiente que tenemos que hacer es identificar qué información es la que nos queremos descargar, y dónde se encuentra dentro del HTML

Todos los elementos de una página tienen un xpath que es unívoco a ese elemento, por lo que puedes usarlo para recuperar la información

- Inspeccionar, botón derecho sobre el elemento, copiar, copiar XPath

<center>
<img src="./imgs/xpath.png"  alt="drawing" width="600"/>
</center>

Cada uno de los resultados tienen la etiqueta li, y la clase list-resultado. Por lo que localizamos todos los elementos que cumplan estas condiciones y los guardamos

<center>
<img src="./imgs/ejemplo_1.png"  alt="drawing" width="600"/>
</center>

Capturamos la ficha técnica de la película (la tabla que está a la derecha)

In [ ]:
datos_a_extraer = soup.find_all('table',{'class' : 'infobox plainlist plainlinks'})

In [ ]:
len(datos_a_extraer)

In [ ]:
type(datos_a_extraer[0])

In [ ]:
tabla_html = datos_a_extraer[0]

In [ ]:
tablas = pd.read_html(str(tabla_html))

In [ ]:
tablas[0]

## 4. Manipulación del contenido

En esta fase vamos iterando sobre lo que hemos conseguido con BeautifulSoup.


Es la primera tabla, por lo que debemos indicar que nos quedamos con únicamente esa tabla

Las filas de la tabla tienen la etiqueta tr, por lo que localizamos todos los elementos que tengan dicha etiqueta, para iterar sobre ellos

In [ ]:
datos_a_extraer = datos_a_extraer[0]
trs = datos_a_extraer.find_all('tr')

Al iterar sobre las filas, queremos extraer los elementos de las columnas de la izquierda th y de la derecha td

Extraemos todos e iteramos sobre ellos para extraer la información

In [ ]:
rows_th = []
rows_td = []

for tr in trs:
    
    ths = tr.find_all('th')
    tds = tr.find_all('td')
    
    for th in ths:
                        
        texto = th.text
        # print(th.text)
        rows_th.append(texto)
        
    for td in tds:
                
        texto = td.text
        #print(td.text)
        rows_td.append(texto)

In [ ]:
resultado = pd.DataFrame(np.column_stack([rows_th, rows_td]))
resultado

Sin embargo, si analizamos el resultado obtenido, no es exáctamente lo que queremos.

La información está descolocada. Y eso es porque hay varios elementos en la tabla que solo están en una de las dos columnas.

Tenemos que identificarlos, y evitar descargarlos

<center>
<img src="./imgs/tabla_mal.png"  alt="drawing" width="600"/>
</center>

In [ ]:
rows_th = []
rows_td = []

for tr in trs:
    
    ths = tr.find_all('th')
    tds = tr.find_all('td')
    
    for th in ths:
        
        if "Ficha técnica" in str(th) or "Datos y cifras" in str(th) or "Compañías" in str(th):
            continue
        
        texto = th.text
        # print(th.text)
        rows_th.append(texto)
        
    for td in tds:
        
        if "Ver todos los créditos" in str(td) or "Ficha" in str(td) or "Wikidata" in str(td):
            continue
        
        texto = td.text
        texto = texto.replace('\n',"")
        #print(td.text)
        rows_td.append(texto)

In [ ]:
resultado = pd.DataFrame(np.column_stack([rows_th, rows_td]))
resultado

Ahora sí que lo hemos conseguido

## Conclusión

El proceso de Web-Scrapping no es un proceso complicado, pero si tedioso.
Y es tedioso porque hay que comprender cuál es la estructura de la web que queremos scrappear y es posible que con el tiempo, un web-scrapper que funcionase, no nos funcione actualmente por qu hayan cambiado la estructura de la web.

- Para asentar conocimientos, vamos a probar a extraer la misma información que ya obtuvimos haciendo este mismo proceso en R
- Los primeros pasos son exáctamente iguales
- Lo único que tenemos que adaptar es la url

## Ejercicios

**8.2.1** De la página: https://www.expansion.com/mercados/cotizaciones/indices/ibex35_I.IB.html obten un daframe con los contenidos de la tabla de cotizaciones.

**8.2.2** Extrae las noticias que aparecen el la web: https://www.expansion.com/mercados/cotizaciones/valores/telefonica_M.TEF.html, Generando un dataframe con la fecha, el título y el resumen de cada noticia.
